In [ ]:
import numpy as np # v1.15.4
import pandas as pd # v0.23.4
from matplotlib import pyplot as plt # v3.0.2
import PIL # v5.3.0
import itertools
from matplotlib.colors import rgb_to_hsv # .3.0.2
import random
import seaborn as sns # v0.9.0
from statsmodels.genmod.generalized_linear_model import GLM # v0.9.0
import statsmodels.api as sm # v0.9.0
from scipy.stats import pearsonr # v1.1.0
import progressbar # v3.39.3

In [ ]:
# Import data files
bilan_ma = pd.read_csv("Bilan-MA_utf8.csv")

plates = ["IMG_070_crop.JPG",
                        "IMG_071_crop.JPG",
                        "IMG_072_crop.JPG",
                        "IMG_073_crop.JPG",
                        "IMG_074_crop.JPG",
                        "IMG_075_crop.JPG",
                        "IMG_076_crop.JPG",
                        "IMG_077_crop.JPG"]

plates_glycerol = ["IMG_098_crop.JPG",
                        "IMG_099_crop.JPG",
                        "IMG_100_crop.JPG",
                        "IMG_101_crop.JPG",
                        "IMG_102_crop.JPG",
                        "IMG_103_crop.JPG",
                        "IMG_104_crop.JPG",
                        "IMG_105_crop.JPG"]

plate_info = pd.read_csv("plate_info.csv", sep=",", index_col=None)
plate_info["strain"] = ["".join((i, str(j))) for i,j in zip(plate_info["cross_letter"].values, plate_info["strain_num"].values)]
# Grid of pixel positions set manually at the center of each colony
xcoords = [306+i*117.8 for i in range(24)]
ycoords = [215+i*117.8 for i in range(16)]

In [ ]:
IMG_SUB={}
background = []
width=10 # half-length (in pixels) of the side of the square extracted for each position
offset=60 # offset (in pixels), to get a grid of background positions intercalated with colonies
top_rank = int(0.1*(2*width)**2) # number of pixels to keep from the square (10%)
RGB = plate_info.copy()
for c in ["R","G","B","H","S","V"]:
    RGB[c]=np.zeros(RGB.shape[0])

for p in plates:
    IMG_SUB[p]={}
    sub = pd.DataFrame(np.zeros([384, 8]), columns=["row","col","R","G","B","H","S","V"])
    sub["row"]=np.tile(range(16), 24)
    sub["col"]=np.repeat(range(24), 16)
    
    image = PIL.Image.open("images/"+p)
    
    with progressbar.ProgressBar(max_value=384) as bar:
        bar_idx=1
        for i, j in itertools.product(range(16), range(24)):
            # Extract a square of pixels for each position
            image_sub = image.crop((xcoords[j]-width ,ycoords[i]-width, xcoords[j]+width, ycoords[i]+width))
            IMG_SUB[p][(i,j)] = image_sub
            # extract the RGB values from the square
            image_sub = pd.DataFrame(np.array(image_sub.getdata())/255, columns=["R","G","B"])

            for c in ["H","S","V"]:
                image_sub[c] = np.zeros(image_sub.shape[0])
            # convert the RGB data in HSV
            for k in image_sub.index:
                image_sub.loc[k, ["H","S","V"]] = rgb_to_hsv(image_sub.loc[k, ["R","G","B"]])
            # Sort the pixels by hue value
            image_sub = image_sub.sort_values(by="H", ascending=False)

            idx = sub.loc[(sub["row"]==i) & (sub["col"]==j)].index[0]
            # Keep the average of the R, G, B, H, S, V values for the top 10% pixels
            sub.loc[idx, "R"] = image_sub.loc[:top_rank, "R"].mean()
            sub.loc[idx, "G"] = image_sub.loc[:top_rank, "G"].mean()
            sub.loc[idx, "B"] = image_sub.loc[:top_rank, "B"].mean()
            sub.loc[idx, "H"] = image_sub.loc[:top_rank, "H"].mean()
            sub.loc[idx, "S"] = image_sub.loc[:top_rank, "S"].mean()
            sub.loc[idx, "V"] = image_sub.loc[:top_rank, "V"].mean()

            #extract the intercalated background position 
            if i>0 and j<23:
                bgd_sub = image.crop((xcoords[j]+offset-width ,ycoords[i]+offset-width, xcoords[j]+offset+width, ycoords[i]+offset+width))
                background.append(np.mean(np.array(bgd_sub.getdata())/255, axis=0))
            bar.update(bar_idx)
            bar_idx+=1
    # add values of the plate to the main table
    RGB.loc[RGB["plate"]==p, ["R","G","B","H","S","V"]] = sub.sort_values(by=["col","row"])[["R","G","B","H","S","V"]].values


# create a separate table for background positions values
background = pd.DataFrame(background, columns=["R","G","B"])
# convert the RGB data in HSV
for c in ["H","S","V"]:
    background[c] = np.zeros(background.shape[0])
for i in background.index:
    background.loc[i, ["H","S","V"]] = rgb_to_hsv(background.loc[i, ["R","G","B"]])

In [ ]:
# Add columns to the RGB table to input colony size data
for c in ["size","circularity","singularity","size_glyc","circularity_glyc","singularity_glyc"]:
    RGB[c]=np.zeros(RGB.shape[0])

# Colony size analysis - YPD
for p in plates:
    # read the image data produced by gitter
    with open("images/"+p+".dat", "r") as fi:
        data = [line.split("\t") for line in fi.read().splitlines() if line[0]!="#"]
    data = pd.DataFrame(data).astype({0:int, 1:int, 2:int}).sort_values(by=[1,0])
    data["circularity"] = ["C" in i for i in data[4].values] # boolean encoding
    data["singularity"] = ["S" in i for i in data[4].values] # boolean encoding
    data[2] = np.log2(data[2].values+1)
    # add values of the plate to the main table
    RGB.loc[RGB["plate"]==p, ["size","circularity","singularity"]] = data[[2,"circularity","singularity"]].values
    
# Colony size analysis - YPG
for p in plates_glycerol:
    # read the image data produced by gitter
    with open("images/"+p+".dat", "r") as fi:
        data = [line.split("\t") for line in fi.read().splitlines() if line[0]!="#"]
    data = pd.DataFrame(data).astype({0:int, 1:int, 2:int}).sort_values(by=[1,0])
    data["circularity"] = ["C" in i for i in data[4].values] # boolean encoding
    data["singularity"] = ["S" in i for i in data[4].values] # boolean encoding
    data[2] = np.log2(data[2].values+1)
    # add values of the plate to the main table
    RGB.loc[RGB["plate_glycerol"]==p, ["size_glyc","circularity_glyc","singularity_glyc"]] = data[[2,"circularity","singularity"]].values
RGB["size_glyc_bin"] = pd.cut(RGB["size_glyc"], 8).values

In [ ]:
# Read RGB and background tables
RGB.to_csv("RGB.csv")
background.to_csv("background.csv")

# Write RGB and background tables
RGB = pd.read_csv("RGB.csv", index_col=0, header=0)
background = pd.read_csv("background.csv", index_col=0, header=0)

In [ ]:
# Filter out contamination and morphology flags
RGB_raw = RGB.copy()
RGB = RGB.drop(RGB.loc[(RGB["contamination"]==0) | (RGB["circularity"]==True) | (RGB["singularity"]==True)].index, axis=0)

# Flag the colonies within [0.02, 0.98] of background hue as background
bgd_hmin, bgd_hmax = [np.quantile(background["H"], x) for x in (0.02, 0.98)]
RGB["bgd"] = [bgd_hmin<=RGB.loc[i,"H"] for i in RGB.index]
# Drop the background positions
RGB = RGB.loc[RGB["bgd"]==False]

In [ ]:
# Plot the raw coloration data
fig, ax = plt.subplots(figsize=[5,5])

sub_fil = RGB.loc[(RGB["contamination"]!=0) & (RGB["circularity"]!=True)& (RGB["singularity"]!=True) & (RGB["bgd"]!=True)].index
sub_out = [i for i in RGB_raw.index if i not in sub_fil]

ax.scatter(RGB_raw.loc[sub_fil, "H"].values, RGB_raw.loc[sub_fil, "S"].values,
           color=list(zip(RGB_raw.loc[sub_fil, "R"].values, RGB_raw.loc[sub_fil, "G"].values, RGB_raw.loc[sub_fil, "B"].values)),
           marker=".")

ax.scatter(RGB_raw.loc[sub_out, "H"].values, RGB_raw.loc[sub_out, "S"].values,
           color=list(zip(RGB_raw.loc[sub_out, "R"].values, RGB_raw.loc[sub_out, "G"].values, RGB_raw.loc[sub_out, "B"].values)),
           marker="x")

ax.set_ylabel("Color saturation")
ax.set_xlabel("Color hue")

plt.tight_layout()
plt.savefig("hue_sat.png", dpi=300)
plt.show()
plt.close()

In [ ]:
# add sporulation ability data to RGB
RGB_spo=pd.DataFrame()
for i in bilan_ma.loc[[not all(np.isnan(i)) for i in bilan_ma[["Fertility_t0","Fertility_tmid","Fertility_tend"]].values]].index:
    s=bilan_ma.loc[i, "Lineage"]
    tmid=bilan_ma.loc[i, "Mid_Glycerol"]
    tend=bilan_ma.loc[i, "Last_Glycerol"]
    
    t0_data = RGB.loc[(RGB["strain"]==s) & (RGB["passage"]==0)].copy()
    t0_data["spo"] = bilan_ma.loc[i, "Fertility_t0"]
    RGB_spo = pd.concat([RGB_spo, t0_data], axis=0)
    
    if tmid=="P16":
        tmid_data = RGB.loc[(RGB["strain"]==s) & (RGB["passage"]==16)].copy()
        tmid_data["spo"] = bilan_ma.loc[i, "Fertility_tmid"]
        RGB_spo = pd.concat([RGB_spo, tmid_data], axis=0)
        
    if tend=="P35":
        tend_data = RGB.loc[(RGB["strain"]==s) & (RGB["passage"]==35)].copy()
        tend_data["spo"] = bilan_ma.loc[i, "Fertility_tend"]
        RGB_spo = pd.concat([RGB_spo, tend_data], axis=0)

RGB_spo["spo_bin"]=[not np.isnan(i) for i in RGB_spo["spo"].values]

In [ ]:
# Fit a GLM with logistic link function to the sporulation ability data
glm_binomial = GLM(RGB_spo["spo_bin"], RGB_spo[["S", "size_glyc"]], family=sm.families.Binomial())
glm_bin_res = glm_binomial.fit()
glm_bin_pvals = glm_bin_res.pvalues
glm_bin_summary = glm_bin_res.summary().tables[1].as_html()
glm_bin_summary = pd.read_html(glm_bin_summary, header=0, index_col=0)[0]
# Pearson's correlation between saturation and glycerol size data
pvals_lin = pearsonr(RGB_spo["S"], RGB_spo["size_glyc"])
# plot
fig, axes = plt.subplots(ncols=3, figsize=[13,4])

ax = axes[0]
sns.boxplot(x="S", y="spo_bin", data=RGB_spo, color="white", ax=ax, order=[True, False], orient="h")
for spo, i in zip([True, False], [0,1]):
    sub=RGB_spo.loc[RGB_spo["spo_bin"]==spo]
    ax.scatter(sub["S"], random.choices(list(np.arange(-0.25,0.25,0.01)+i), k=sub.shape[0]), 
              color=list(zip(sub["R"].values, sub["G"].values, sub["B"].values, np.repeat(1, sub.shape[0]))))
ax.set_xlabel("Saturation")
ax.set_ylabel("Sporulation capacity")
ax.set_yticklabels(["Yes","No"])
ax.set_title("z=%.3f [%.3f, %.3f]\nP-value = %.3f" % tuple(list(glm_bin_summary.loc["S",["z","[0.025","0.975]"]].values)+[glm_bin_pvals["S"]]))



ax = axes[1]

for spo, i in zip([True, False], [0,1]):
    sub=RGB_spo.loc[RGB_spo["spo_bin"]==spo]
    ax.scatter(sub["size_glyc"], random.choices(list(np.arange(-0.25,0.25,0.01)+i), k=sub.shape[0]), 
              color=list(zip(sub["R"].values, sub["G"].values, sub["B"].values, np.repeat(1, sub.shape[0]))))
sns.boxplot(x="size_glyc", y="spo_bin", data=RGB_spo, color="white", ax=ax, order=[True, False], orient="h")
ax.set_xlabel("Glycerol growth")
ax.set_ylabel("Sporulation capacity")
ax.set_yticklabels(["Yes","No"])
ax.set_title("z=%.3f [%.3f, %.3f]\nP-value = %.3e" % tuple(list(glm_bin_summary.loc["size_glyc",["z","[0.025","0.975]"]].values)+[glm_bin_pvals["size_glyc"]]))

ax = axes[2]
ax.scatter(RGB_spo["S"], RGB_spo["size_glyc"], marker="o",
           color=list(zip(RGB_spo["R"].values, RGB_spo["G"].values, RGB_spo["B"].values)))

ax.set_ylabel("Glycerol growth")
ax.set_xlabel("Saturation")
ax.set_title("r = %.3f P-value = %.3e" % pvals_lin)

plt.tight_layout()
plt.savefig("spo_ability_sat_glyc.png", dpi=300)
plt.show()
plt.close()

In [ ]:
# Assess the correlation between sporulation ability loss and variation in 
# red saturation or growth on glycerol

spo_corr = []

for s in set(RGB["strain"]):
    idx=bilan_ma.loc[bilan_ma["Lineage"]==s].index[0]
    
    #Iterate the different timing scenarios for sporulation ability loss
    
    # 1) sporulation ability loss occurred between Tini and Tmid
    if all([np.isnan(bilan_ma.loc[idx, "Fertility_tend"]),
            np.isnan(bilan_ma.loc[idx, "Fertility_tmid"]),
            (not np.isnan(bilan_ma.loc[idx, "Fertility_t0"])),
           bilan_ma.loc[idx, "Last_Glycerol"]!="P4"]):

        tmid = int(bilan_ma.loc[idx, "Mid_Glycerol"][1:])

        sub_rgb = RGB.loc[RGB["strain"]==s].copy()
        sub_rgb.index = sub_rgb["passage"].values
        # Iterate through the possible timepoint pairs for which there can be data
        for (t0, t1) in [(0,35), (0,16), (0,1)]:
            if all([x in sub_rgb.index for x in (t0, t1)]) and t1>=tmid: 
                tp="success"
                break
            else:
                tp="fail"
                continue

        if tp!="fail":
            glycerol_diff=sub_rgb.loc[t1, "size_glyc"]-sub_rgb.loc[t0, "size_glyc"]
            hue_diff=sub_rgb.loc[t1, "H"]-sub_rgb.loc[t0, "H"]
            loss=1
            spo_corr.append([s, t0, t1, loss, "Tini-Tmid", glycerol_diff, hue_diff])
                    
    # 2) fertility loss occurred between Tmid and Tend
    if all([np.isnan(bilan_ma.loc[idx, "Fertility_tend"]),
            (not np.isnan(bilan_ma.loc[idx, "Fertility_tmid"])),
            (not np.isnan(bilan_ma.loc[idx, "Fertility_t0"])),
           bilan_ma.loc[idx, "Last_Glycerol"]!="P4"]):
        
        tmid = int(bilan_ma.loc[idx, "Mid_Glycerol"][1:])
        tend = int(bilan_ma.loc[idx, "Last_Glycerol"][1:])
        
        sub_rgb = RGB.loc[RGB["strain"]==s].copy()
        sub_rgb.index = sub_rgb["passage"].values
        # Iterate through the possible timepoint pairs for which there can be data
        for (t0, t1) in [(0,35), (1,35), (0,16), (1,16), (0,1), (16,35)]:
            if all([x in sub_rgb.index for x in (t0, t1)]) and t1>=tend and t0<=tmid: 
                tp="success"
                break
            else:
                tp="fail"
                continue

        if tp!="fail":
            glycerol_diff=sub_rgb.loc[t1, "size_glyc"]-sub_rgb.loc[t0, "size_glyc"]
            sat_diff=sub_rgb.loc[t1, "S"]-sub_rgb.loc[t0, "S"]
            loss=1
            spo_corr.append([s, t0, t1, loss, "Tmid-Tend", glycerol_diff, sat_diff])
            
    # 3) fertility is never lost
    if all([not(np.isnan(bilan_ma.loc[idx, "Fertility_tend"])),
            (not np.isnan(bilan_ma.loc[idx, "Fertility_tmid"])),
            (not np.isnan(bilan_ma.loc[idx, "Fertility_t0"]))]):
        
        sub_rgb = RGB.loc[RGB["strain"]==s].copy()
        sub_rgb.index = sub_rgb["passage"].values
        # Iterate through the possible timepoint pairs for which there can be data
        for (t0, t1) in [(0,35), (1,35), (0,16), (1,16), (0,1), (16,35)]:
            if all([x in sub_rgb.index for x in (t0, t1)]): 
                tp="success"
                break
            else:
                tp="fail"
                continue

        if tp!="fail":
            glycerol_diff=sub_rgb.loc[t1, "size_glyc"]-sub_rgb.loc[t0, "size_glyc"]
            sat_diff=sub_rgb.loc[t1, "S"]-sub_rgb.loc[t0, "S"]
            loss=0
            spo_corr.append([s, t0, t1, loss, "None", glycerol_diff, sat_diff])
            
spo_corr = pd.DataFrame(spo_corr, columns=["strain","t0","t1","loss","loss_range","glycerol","sat"])

In [ ]:
glm_binomial = GLM(spo_corr["loss"], spo_corr[["sat", "glycerol"]], family=sm.families.Binomial())
glm_bin_res = glm_binomial.fit()
glm_bin_pvals = glm_bin_res.pvalues
glm_bin_summary = glm_bin_res.summary().tables[1].as_html()
glm_bin_summary = pd.read_html(glm_bin_summary, header=0, index_col=0)[0]

pvals_lin = pearsonr(spo_corr["sat"], spo_corr["glycerol"])

fig, axes = plt.subplots(ncols=3, figsize=[13,4])

ax = axes[0]
sns.boxplot(x="sat", y="loss", data=spo_corr, color="white", ax=ax, order=[0,1], orient="h")
for spo in [0,1]:
    sub=spo_corr.loc[spo_corr["loss"]==spo]
    ax.scatter(sub["sat"], random.choices(list(np.arange(-0.25,0.25,0.01)+spo), k=sub.shape[0]),
              color="black", marker="o")
ax.set_xlabel(r"$\Delta$ Saturation")
ax.set_ylabel("Sporulation capacity loss")
ax.set_yticklabels(["No", "Yes"])
ax.set_title("z=%.3f [%.3f, %.3f]\nP-value = %.3f" % tuple(list(glm_bin_summary.loc["sat",["z","[0.025","0.975]"]].values)+[glm_bin_pvals["sat"]]))



ax = axes[1]
sns.boxplot(x="glycerol", y="loss", data=spo_corr, color="white", ax=ax, order=[0,1], orient="h")

for spo in [0,1]:
    sub=spo_corr.loc[spo_corr["loss"]==spo]
    ax.scatter(sub["glycerol"], random.choices(list(np.arange(-0.25,0.25,0.01)+spo), k=sub.shape[0]), 
              color="black", marker="o")
ax.set_xlabel(r"$\Delta$ Glycerol growth")
ax.set_ylabel("Sporulation capacity loss")
ax.set_yticklabels(["No", "Yes"])
ax.set_title("z=%.3f [%.3f, %.3f]\nP-value = %.3f" % tuple(list(glm_bin_summary.loc["glycerol",["z","[0.025","0.975]"]].values)+[glm_bin_pvals["glycerol"]]))

ax = axes[2]
ax.scatter(spo_corr["sat"], spo_corr["glycerol"], marker="o",
           color="black")

ax.set_ylabel(r"$\Delta$ Glycerol growth")
ax.set_xlabel(r"$\Delta$ Saturation")
ax.set_title("r = %.3f P-value = %.3e" % pvals_lin)

plt.tight_layout()
plt.savefig("spo_loss_sat_glyc_diff.png", dpi=300)
plt.show()
plt.close()